In [1]:
import pandas as pd

### 2022 source file spec

In [2]:
# Define the file path
file_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\raw\kdkf_2022_raw.xlsx'

# Load the Excel file to get sheet names
xls = pd.ExcelFile(file_path)

# Define the years
previous_year = 2021
start_year = 2022
target_year = 2023

# Set the pandas option to display floating point numbers using the decimal format
pd.options.display.float_format = '{:.5f}'.format

### Master Loop

In [3]:
# Initialize an empty DataFrame for the master data
master_2022 = pd.DataFrame()

# Loop over each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Check if the sheet name starts with "HRM2_KT_"
    if sheet_name.startswith("HRM2_KT_"):
        # Load the sheet into a DataFrame, skipping the first three rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=3)

        # Merge the first row with the column headers
        df.columns = df.iloc[0, :2].tolist() + (df.columns[2:]).tolist()
        df = df.iloc[1:].reset_index(drop=True)

        # Update Headers of the df
        rename_dict = {
            'HRM 2': 'HRM2-ID',
            'Unnamed: 2': 'HRM2-Bezeichnung',
            'Rechnung': f'Rechnung {previous_year}',
            'Budget*': f'Budget {start_year}',
            'Anpassung Budget**': f'Anpassung Budget {start_year}',
            'Budgetplus***': f'Budgetplus {start_year}',
            'Rechnung.1': f'Rechnung {start_year}',
            'Budget': f'Budget {target_year}'
        }
        df.rename(columns=rename_dict, inplace=True)

        # Filter the DataFrame to keep only rows where 'Referenz-ID' contains 'HRM2'
        df = df[df['Referenz-ID'].str.contains('HRM2', na=False)]

        # Add a new column with the DataFrame name (sheet name) and the year spec
        df['Source'] = sheet_name
        df['Kanton'] = df['Source'].str.split('HRM2_KT_').str[1]
        df['Year'] = 2022

        # Use pd.concat to add the DataFrame to the master DataFrame
        master_2022 = pd.concat([master_2022, df], ignore_index=True)

# Display the first few rows of the master DataFrame
master_2022.head()

,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2021,Budget 2022,Anpassung Budget 2022,Budgetplus 2022,Rechnung 2022,Budget 2023,Source,Kanton,Year,Compte,Adaptation Budget**,Compte.1
0,HRM2_ER0030,30,Personalaufwand,26152620.02857,26630564.07622,308222.46998,26938786.54620,26985594.56335,28032655.05247,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
1,HRM2_ER0031,31,Sach- und übriger Betriebsaufwand,10287100.48293,10415701.98070,648662.04550,11064364.02620,10806538.02481,10798381.31853,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
2,HRM2_ER0314,davon 314,baulicher und betrieblicher Unterhalt,1138027.15383,1111854.51800,19209.29500,1131063.81300,1118726.41120,1149341.78500,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
3,HRM2_ER0318,davon 3180,Wertberichtigungen auf Forderungen,753.92456,52483.26600,3469.80000,55953.06600,108041.00892,60863.81000,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
4,HRM2_neu_ER0033,33,Abschreibungen VV,2829988.73580,2898268.65011,5258.40000,2903527.05011,2839873.69033,2926517.17731,HRM2_KT_alle,alle,2022,NaN,NaN,NaN


### Translating FR columns to DE

In [4]:
# # Display rows where the column 'Kanton' is 'HRM2_KT_VD'
# filtered_rows = master_2022[master_2022['Kanton'] == 'HRM2_KT_VD']

# # Display the filtered rows
# filtered_rows.head()

In [5]:
# Update 'master_2022' DataFrame as per the specified conditions

# 1. Update 'Rechnung 2021' based on 'Compte'
master_2022['Rechnung 2021'] = master_2022.apply(
    lambda row: row['Compte'] if pd.isna(row['Rechnung 2021']) and not pd.isna(row['Compte']) else row['Rechnung 2021'],
    axis=1
)

# 2. Update 'Anpassung Budget 2022' based on 'Adaptation Budget**'
master_2022['Anpassung Budget 2022'] = master_2022.apply(
    lambda row: row['Adaptation Budget**'] if pd.isna(row['Anpassung Budget 2022']) and not pd.isna(row['Adaptation Budget**']) else row['Anpassung Budget 2022'],
    axis=1
)

# 3. Update 'Rechnung 2022' based on 'Compte.1'
master_2022['Rechnung 2022'] = master_2022.apply(
    lambda row: row['Compte.1'] if pd.isna(row['Rechnung 2022']) and not pd.isna(row['Compte.1']) else row['Rechnung 2022'],
    axis=1
)

# Display the first few rows of the updated DataFrame
master_2022.head()


,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2021,Budget 2022,Anpassung Budget 2022,Budgetplus 2022,Rechnung 2022,Budget 2023,Source,Kanton,Year,Compte,Adaptation Budget**,Compte.1
0,HRM2_ER0030,30,Personalaufwand,26152620.02857,26630564.07622,308222.46998,26938786.54620,26985594.56335,28032655.05247,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
1,HRM2_ER0031,31,Sach- und übriger Betriebsaufwand,10287100.48293,10415701.98070,648662.04550,11064364.02620,10806538.02481,10798381.31853,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
2,HRM2_ER0314,davon 314,baulicher und betrieblicher Unterhalt,1138027.15383,1111854.51800,19209.29500,1131063.81300,1118726.41120,1149341.78500,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
3,HRM2_ER0318,davon 3180,Wertberichtigungen auf Forderungen,753.92456,52483.26600,3469.80000,55953.06600,108041.00892,60863.81000,HRM2_KT_alle,alle,2022,NaN,NaN,NaN
4,HRM2_neu_ER0033,33,Abschreibungen VV,2829988.73580,2898268.65011,5258.40000,2903527.05011,2839873.69033,2926517.17731,HRM2_KT_alle,alle,2022,NaN,NaN,NaN


In [6]:
len(master_2022)

5319

In [7]:
# # Display rows where the column 'Kanton' is 'HRM2_KT_VD'
# filtered_rows = master_2022[master_2022['Kanton'] == 'HRM2_KT_VD']

# # Display the filtered rows
# filtered_rows.head()

### Save df to .csv

In [8]:
# Define the path where the CSV file will be saved
output_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\ready\kdkf_2022_data.csv'

# Save the DataFrame to a CSV file
master_2022.to_csv(output_path, index=False)
